In [1]:
import pandas as pd
import numpy as np
import os
import pycocotools.mask as mutils
import base64
import numpy as np
from pycocotools import mask as coco_mask_d
import typing as t
import zlib
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm

DATA_DIR = '/mnt/chicm/data/open-images/segmentation'

In [2]:
def get_top_classes(start_index, end_index, class_file='top_classes_level1.csv'):
    df = pd.read_csv(os.path.join(DATA_DIR, class_file))
    c = df['class'].values[start_index:end_index]
    #print(df.head())
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi


In [3]:
level1_classes = get_top_classes(0, 275)[0]
parent_classes = get_top_classes(0, 25, 'top_classes_parent.csv')[0]

seg_classes = set(level1_classes) | set(parent_classes)
len(seg_classes)

300

In [4]:
df = pd.read_csv('../sub_od627_275_softnms_top100.csv')
df.PredictionString = df.PredictionString.fillna('')
df.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/083wq 0.0008238 eNp1VNtyozAM/SWbSwL4YbczBjc...
1,52a76f3fe21185bf,1024,683,/m/04dr76w 0.0057520 eNqVUmlvwjAM/UtO0nRsDBAdk...
2,65e9da7c04c8e4cd,1024,682,/m/04yx4 0.0002475 eNoLCIhKNQAABN0BkA== /m/079...
3,6d10fa21d93bd6cc,1024,682,/m/05r655 0.0169685 eNrtUe1OwzAMfKV+J1mRJkRXN2...
4,0841dee573a875ca,1024,683,/m/05r655 0.0010319 eNptUu9PgzAQ/ZcOCszFmPhBox...


In [5]:
TOPK = 50
def get_topk_preds(pred_str):
    try:
        if len(pred_str) < 1:
            return ''
    except:
        print('pred_str:', pred_str)
        raise
    det_items = pred_str.split()
    dets, det = [], []
    for i in range(len(det_items)):
        det.append(det_items[i])
        if (i+1) % 3 == 0:
            assert '/' in det[0]
            if det[0] in seg_classes:
                dets.append(det)
            det = []
    top_dets = sorted(dets, key=lambda x: float(x[1]), reverse=True)[:TOPK]
    
    res = []
    for d in top_dets:
        for di in d:
            res.append(di)
    return ' '.join(res)

In [6]:
#sorted(ens_det, key=lambda x: x[2], reverse=True)[:MAX_NUM]

In [7]:
df.PredictionString = df.PredictionString.map(lambda x: get_topk_preds(x))
df.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/01k6s3 0.6238899 eNqNU01TgzAQ/UvZBEqJnhyphU...
1,52a76f3fe21185bf,1024,683,/m/03y6mg 0.3265402 eNrNUl1XwjAM/Utpt3WIqMeDA1...
2,65e9da7c04c8e4cd,1024,682,/m/025nd 0.0004185 eNqtlsl24jAQRX+p5N51sulz6GV...
3,6d10fa21d93bd6cc,1024,682,/m/01xyhv 0.9648581 eNptUttugzAM/aWQkLBpUx82aW...
4,0841dee573a875ca,1024,683,/m/0342h 0.9437939 eNptUdtOwzAM/SUnvYwJhBCgial...


In [9]:
df.to_csv('../sub_od627_275_softnms_top50.csv', index=False)